In [1]:
from pandas import read_csv
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import statistics
from statsmodels.tsa.stattools import adfuller, kpss, acf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

In [2]:
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

In [3]:
mape_list = []
rmse_list = []
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    mape_list.append(mape)
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    rmse_list.append(rmse)
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 
            'corr':corr})

# Часть 1 - 1й датасет

## 1s timestamp

In [111]:
num = [str(i) for i in range(22)]

In [112]:
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P2/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P3/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q1/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q2/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q3/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U1/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U2/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U3/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

In [113]:
def prepare_data(df, num_elements=50):
    new_data = []
    length = df.shape[0]
    current_index = 0
    step = int(0.2 * num_elements)
    while(current_index <= length - num_elements):
        new_data.append(df[current_index:current_index+num_elements])
        current_index += step
    return new_data

In [114]:
p1 = [prepare_data(df) for df in p1]
p1 = sum(p1, [])

p2 = [prepare_data(df) for df in p2]
p2 = sum(p2, [])

p3 = [prepare_data(df) for df in p3]
p3 = sum(p3, [])

q1 = [prepare_data(df) for df in q1]
q1 = sum(q1, [])

q2 = [prepare_data(df) for df in q2]
q2 = sum(q2, [])

q3 = [prepare_data(df) for df in q3]
q3 = sum(q3, [])

u1 = [prepare_data(df) for df in u1]
u1 = sum(u1, [])

u2 = [prepare_data(df) for df in u2]
u2 = sum(u2, [])

u3 = [prepare_data(df) for df in u3]
u3 = sum(u3, [])

In [115]:
print([i for i in range(2, 7)])

[2, 3, 4, 5, 6]


In [116]:
def train_arima(df, index=0, max_m=2):
#     train, test = train_test_split(df, test_size=0.2, shuffle=False)
    train = df[:-10]
    test = df[-10:]
    best_model = pm.auto_arima(train.values, start_p=0, start_q=0,
                               test='adf',       # use adftest to find optimal 'd'
                               max_p=3, max_q=3, # maximum p and q
                               m=1,              # frequency of series
                               d=None,           # let model determine 'd'
                               seasonal=True,   # Seasonality
                               start_P=0, 
                               D=1, 
                               trace=False,
                               error_action='ignore',  
                               suppress_warnings=True, 
                               stepwise=True)
    for i in range(2, max_m):
        model = pm.auto_arima(train.values, start_p=0, start_q=0,
                          test='adf',       # use adftest to find optimal 'd'
                          max_p=3, max_q=3, # maximum p and q
                          m=i,              # frequency of series
                          d=None,           # let model determine 'd'
                          seasonal=True,   # Seasonality
                          start_P=0, 
                          D=1, 
                          trace=False,
                          error_action='ignore',  
                          suppress_warnings=True, 
                          stepwise=True)
        
        # Find the best model with MSE
        if np.mean((model.predict(n_periods=test.shape[0]) - test.values)**2) < np.mean((best_model.predict(n_periods=test.shape[0]) - test.values)**2):
            best_model = model
#     print("Data " + str(index))
#     print(forecast_accuracy(model.predict(n_periods=test.shape[0]), test.values))
    forecast_accuracy(best_model.predict(n_periods=test.shape[0]), test.values)
#     print("\n")

In [117]:
for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [118]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.022022992750171097
15.069444416590926


In [35]:
# train_multi = pd.concat([p1[0], p2[0], p3[0], q1[0], q2[0], q3[0], u1[0]], axis=1)

In [36]:
# train, test = train_test_split(train_multi, test_size=0.2, shuffle=False)

In [37]:
# from statsmodels.tsa.statespace.varmax import VARMAX

# model = VARMAX(train, order = (1,1))
# model_fit = model.fit()
# print(forecast_accuracy(model_fit.forecast(steps=test.shape[0]), test.values))

In [38]:
# model_fit.forecast(steps=test.shape[0])

In [15]:
# model = ARIMA(train, order=(0, 1, 0))  
# fitted = model.fit(disp=-1)  

# # Forecast
# fc, se, conf = fitted.forecast(len(test), alpha=0.05)  # 95% conf

# # Make as pandas series
# fc_series = pd.Series(fc, index=test.index)
# lower_series = pd.Series(conf[:, 0], index=test.index)
# upper_series = pd.Series(conf[:, 1], index=test.index)
# print(fitted.summary())

# # Plot
# plt.figure(figsize=(12,5), dpi=100)
# plt.plot(train, label='training')
# plt.plot(test, label='actual')
# plt.plot(fc_series, label='forecast')
# plt.fill_between(lower_series.index, lower_series, upper_series, 
#                  color='k', alpha=.15)
# plt.title('Forecast vs Actuals')
# plt.legend(loc='upper left', fontsize=8)
# plt.show()

## 1.5s timestamp

In [32]:
num = [str(i) for i in range(141)]

In [33]:
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P2/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P3/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q1/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q2/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q3/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U1/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U2/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U3/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

In [34]:
p1 = [prepare_data(df) for df in p1]
p1 = sum(p1, [])

p2 = [prepare_data(df) for df in p2]
p2 = sum(p2, [])

p3 = [prepare_data(df) for df in p3]
p3 = sum(p3, [])

q1 = [prepare_data(df) for df in q1]
q1 = sum(q1, [])

q2 = [prepare_data(df) for df in q2]
q2 = sum(q2, [])

q3 = [prepare_data(df) for df in q3]
q3 = sum(q3, [])

u1 = [prepare_data(df) for df in u1]
u1 = sum(u1, [])

u2 = [prepare_data(df) for df in u2]
u2 = sum(u2, [])

u3 = [prepare_data(df) for df in u3]
u3 = sum(u3, [])

In [35]:
for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1620: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1620: RuntimeWarning: divide by zero encountered in dou

In [36]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.12269054258899613
220.7941096257094


## 2s timestamp

In [203]:
num = [str(i) for i in range(902)]

In [204]:
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P2/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P3/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q1/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q2/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q3/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U1/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U2/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U3/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

In [205]:
p1 = [prepare_data(df) for df in p1]
p1 = sum(p1, [])

p2 = [prepare_data(df) for df in p2]
p2 = sum(p2, [])

p3 = [prepare_data(df) for df in p3]
p3 = sum(p3, [])

q1 = [prepare_data(df) for df in q1]
q1 = sum(q1, [])

q2 = [prepare_data(df) for df in q2]
q2 = sum(q2, [])

q3 = [prepare_data(df) for df in q3]
q3 = sum(q3, [])

u1 = [prepare_data(df) for df in u1]
u1 = sum(u1, [])

u2 = [prepare_data(df) for df in u2]
u2 = sum(u2, [])

u3 = [prepare_data(df) for df in u3]
u3 = sum(u3, [])

In [206]:
for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [207]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.06731475198331945
71.88725322363149


## 5s timestamp

In [208]:
num = [str(i) for i in range(19)]

In [209]:
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P2/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P3/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q1/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q2/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q3/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U1/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U2/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U3/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

In [210]:
p1 = [prepare_data(df) for df in p1]
p1 = sum(p1, [])

p2 = [prepare_data(df) for df in p2]
p2 = sum(p2, [])

p3 = [prepare_data(df) for df in p3]
p3 = sum(p3, [])

q1 = [prepare_data(df) for df in q1]
q1 = sum(q1, [])

q2 = [prepare_data(df) for df in q2]
q2 = sum(q2, [])

q3 = [prepare_data(df) for df in q3]
q3 = sum(q3, [])

u1 = [prepare_data(df) for df in u1]
u1 = sum(u1, [])

u2 = [prepare_data(df) for df in u2]
u2 = sum(u2, [])

u3 = [prepare_data(df) for df in u3]
u3 = sum(u3, [])

In [211]:
for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [212]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.25926939792907205
310.7907965778942


## 15s timestamp

In [213]:
num = [str(i) for i in range(20)]

In [214]:
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P2/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P3/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q1/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q2/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q3/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U1/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U2/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U3/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

In [215]:
p1 = [prepare_data(df) for df in p1]
p1 = sum(p1, [])

p2 = [prepare_data(df) for df in p2]
p2 = sum(p2, [])

p3 = [prepare_data(df) for df in p3]
p3 = sum(p3, [])

q1 = [prepare_data(df) for df in q1]
q1 = sum(q1, [])

q2 = [prepare_data(df) for df in q2]
q2 = sum(q2, [])

q3 = [prepare_data(df) for df in q3]
q3 = sum(q3, [])

u1 = [prepare_data(df) for df in u1]
u1 = sum(u1, [])

u2 = [prepare_data(df) for df in u2]
u2 = sum(u2, [])

u3 = [prepare_data(df) for df in u3]
u3 = sum(u3, [])

In [216]:
for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [217]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.15609903407231004
185.96882724302415


### length 100

In [218]:
num = [str(i) for i in range(22)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

In [219]:
p1 = [prepare_data(df, 100) for df in p1]
p1 = sum(p1, [])

In [220]:
for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [221]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.022727424819621245
17.917543144597257


In [222]:
num = [str(i) for i in range(141)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 100) for df in p1]
p1 = sum(p1, [])

In [223]:
for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [224]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.0034973447556556863
3.611298909345747


In [225]:
num = [str(i) for i in range(902)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 100) for df in p1]
p1 = sum(p1, [])

In [226]:
for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [227]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.04305993769981409
24.54895004528533


In [228]:
num = [str(i) for i in range(19)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 100) for df in p1]
p1 = sum(p1, [])

In [229]:
for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [230]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.1113332404242501
154.17675845118927


In [231]:
num = [str(i) for i in range(20)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 100) for df in p1]
p1 = sum(p1, [])

In [232]:
for df in p1[:100]:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [233]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.12367809851197145
157.56251980784566


### length 150

In [ ]:
def print_and_clear_error():
    print(sum(mape_list) / len(mape_list))
    print(sum(rmse_list) / len(rmse_list))
    mape_list = []
    rmse_list = []

In [234]:
num = [str(i) for i in range(22)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 150) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [235]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.039849736891657585
32.29903667103372


In [236]:
num = [str(i) for i in range(141)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 150) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [237]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.0032503743219336385
3.3381529886125123


In [238]:
num = [str(i) for i in range(902)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 150) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [239]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.02772063903912214
10.919837078775918


In [240]:
num = [str(i) for i in range(19)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 150) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [241]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.07907836305885577
98.2610209568289


In [242]:
num = [str(i) for i in range(20)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 150) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [243]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.1224501345217447
74.75966817515054


### length 200

In [244]:
num = [str(i) for i in range(22)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 200) for df in p1]
p1 = sum(p1, [])

for df in p1[:100]:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [245]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.02087464227267879
11.685957553860714


In [246]:
num = [str(i) for i in range(141)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 200) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [247]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.03131215241109015
61.483631726788474


In [248]:
num = [str(i) for i in range(902)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 200) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [249]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.026174339888765955
16.88541540139596


In [250]:
num = [str(i) for i in range(19)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 200) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [251]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.04670491052907657
74.9555393161814


In [252]:
num = [str(i) for i in range(20)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 200) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [253]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.1504878533437745
137.33889271736243


### length 250

In [254]:
num = [str(i) for i in range(22)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 250) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [255]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.018042233729265188
6.372637008992316


In [256]:
num = [str(i) for i in range(141)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 250) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [257]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.020490041786970017
54.03695749714545


In [258]:
num = [str(i) for i in range(902)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 250) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [259]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.039300814184051826
31.228850467526016


In [260]:
num = [str(i) for i in range(19)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 250) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [261]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.06969356797673282
91.01306563599196


In [262]:
num = [str(i) for i in range(20)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 250) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df, max_m=3)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [263]:
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.13772887510994072
135.56045486807156


### length 300

In [128]:
def learn(all_num, sample_size, ts, max_m=2):
    num = [str(i) for i in range(all_num)]
    p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/' + str(ts) + '/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

    p1 = [prepare_data(df, sample_size) for df in p1]
    p1 = sum(p1, [])
    
    if not p1:
        print("Not enough data")
        return
    else:
        print("Data count: ", len(p1))

    for df in p1:
        train_arima(df, max_m=max_m)

In [195]:
learn(22, 300, 1000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  13


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.035870412086767285
56.43955308823243


In [196]:
learn(141, 300, 1500, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.01926614083443493
39.46309774023281


In [197]:
learn(902, 300, 2000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100
0.01994701891743858
6.680278415061001


In [198]:
learn(19, 300, 5000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.07585227590294856
92.44993776740517


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [199]:
learn(20, 300, 15000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.1369425467941746
113.59775205208776


### length 350

In [194]:
learn(22, 350, 1000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  10


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.022597912606074105
10.841523250266091


In [193]:
learn(141, 350, 1500, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  74


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.032981399191221894
61.19566371980655


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [192]:
learn(902, 350, 2000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.031054686855860842
23.643119968866202


In [191]:
learn(19, 350, 5000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.06942045633809048
109.1191827797553


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [190]:
learn(20, 350, 15000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.13881445038407147
186.8076532808388


### length 400

In [185]:
learn(22, 400, 1000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  8
0.03163962265310055
24.843526368431263


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [186]:
learn(141, 400, 1500, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  51


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.02676593241182609
57.79346763601705


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [187]:
learn(902, 400, 2000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.01955951972704367
4.569266561187038


In [188]:
learn(19, 400, 5000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.07017125241896327
88.38521334215481


In [189]:
learn(20, 400, 15000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.10262465624338908
99.86914948198844


### length 450

In [121]:
learn(22, 450, 1000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  7
0.03669594245786824
98.14901986869755


In [124]:
learn(141, 450, 1500, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  33


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.011561833899471796
21.331805250661287


In [125]:
learn(902, 450, 2000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  1163


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.024711360850436414
11.165178221016053


In [126]:
learn(19, 450, 5000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  262


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.04145872911701352
57.721586030043134


In [127]:
learn(20, 450, 15000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  143


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.08035862811586875
67.59561592412925


### length 500

In [180]:
learn(22, 500, 1000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  6
0.015062317455243439
6.763239047307859


In [181]:
learn(141, 500, 1500, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  20
0.03410558717724972
45.17720891062846


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [182]:
learn(902, 500, 2000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.02060325216972063
9.388391500007003


In [183]:
learn(19, 500, 5000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.08020289495601222
114.59409843671362


In [184]:
learn(20, 500, 15000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.11531337992502934
123.56388937758337


### length 550

In [175]:
learn(22, 550, 1000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  5
0.04463562137356567
25.430146401518947


In [176]:
learn(141, 550, 1500, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  11
0.0355371652248534
74.91808167313604


In [177]:
learn(902, 550, 2000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.01210996469680587
2.9714981264859364


In [178]:
learn(19, 550, 5000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.07003773028076056
80.47490229555426


In [179]:
learn(20, 550, 15000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.10606254322361561
143.31419893839433


### length 600

In [170]:
learn(22, 600, 1000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  4
0.010736031664134872
6.163095228376584


In [171]:
learn(141, 600, 1500, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  6
0.00373271140397571
7.631813086114083


In [172]:
learn(902, 600, 2000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.01030306583050219
2.816851996090661


In [173]:
learn(19, 600, 5000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.04884383050116992
69.79186013502158


In [174]:
learn(20, 600, 15000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  90


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.07873925507633427
66.200673358604


### length 650

In [165]:
learn(22, 650, 1000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  3
0.05194511364698492
11.050479652116204


In [166]:
learn(141, 650, 1500, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  5
0.13484705867058006
352.03248657135714


In [167]:
learn(902, 650, 2000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100
0.021326393501111766
20.545770955130386


In [168]:
learn(19, 650, 5000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.07813178806192428
111.70450991980674


In [169]:
learn(20, 650, 15000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  77


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.0920424834829471
124.38087722786639


### length 700

In [138]:
learn(22, 700, 1000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  3
0.01478105691739115
4.020914273017367


In [139]:
learn(141, 700, 1500, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  3
0.007259358457681379
15.75464041870804


In [140]:
learn(902, 700, 2000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.019860971674224567
8.791518296166226


In [141]:
learn(19, 700, 5000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.0437956822447283
60.44782950351768


In [142]:
learn(20, 700, 15000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  67


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.09553607616050915
94.55385085660873


### length 750

In [143]:
learn(22, 750, 1000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  2
0.00916713425927623
3.954813505081606


In [144]:
learn(141, 750, 1500, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  2
0.0032522572298014103
5.711140199475415


In [145]:
learn(902, 750, 2000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100
0.01266566915449856
2.29328346007472


In [146]:
learn(19, 750, 5000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.06841256787590083
91.08970113841063


In [147]:
learn(20, 750, 15000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  59
0.059024923736249756
68.8023527631057


### length 800

In [148]:
learn(22, 800, 1000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  2
0.01714596079937446
2.737026288607536


In [149]:
learn(141, 800, 1500, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  2
0.277854103743011
306.4679874819427


In [150]:
learn(902, 800, 2000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100
0.010575896574341603
2.278981009610269


In [151]:
learn(19, 800, 5000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.04185443773707624
55.4847852103786


In [152]:
learn(20, 800, 15000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  51


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.08987890598459856
68.343040306623


### length 850

In [153]:
learn(22, 850, 1000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  1
0.012705258474515781
1.4794812206333345


In [154]:
learn(902, 850, 2000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100
0.01587470534254466
3.4955810294204657


In [155]:
learn(19, 850, 5000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100
0.05042899822349947
55.632558182301466


In [156]:
learn(20, 850, 15000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  44


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.055229653999954526
24.759969130713582


### length 900

In [157]:
learn(22, 900, 1000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  1
0.007859268589238032
7.26707333255691


In [158]:
learn(902, 900, 2000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.012945040667597218
2.9808671137933374


In [159]:
learn(19, 900, 5000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  97


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.05568085363009367
67.36429851352722


In [160]:
learn(20, 900, 15000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  39


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.06715778479713885
40.2869434243135


### length 950

In [161]:
learn(22, 950, 1000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  1
0.0020212212469643553
1.7809878225317233


In [162]:
learn(902, 950, 2000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.017709490481336965
6.494154300624287


In [163]:
learn(19, 950, 5000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  88


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.044050042333258876
75.87447634590218


In [164]:
learn(20, 950, 15000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  32
0.06207865696872941
51.891121950369204


### length 1000

In [132]:
learn(22, 1000, 1000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  1
0.002997596322281956
2.8061415491450616


In [135]:
learn(902, 1000, 2000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  100
0.014141253892614667
3.3926941836427655


In [136]:
learn(19, 1000, 5000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  80


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.04840166171978579
72.35612567155965


In [137]:
learn(20, 1000, 15000, 3)
rmse_list = [el for ind, el in enumerate(rmse_list) if mape_list[ind] < 1]
mape_list = [el for ind, el in enumerate(mape_list) if el < 1] # удалим куски датасетов, на которых алгоритм не сошелся

print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  30
0.06687608787512529
34.9672491270022
